# LangChain 1-hour Sprint

![LangChain Logo](https://github.com/rastringer/promptcraft_notebooks/blob/main/images/langchain.png?raw=1)

LangChain is a framework for developing applications infused with LLM magic. In this notebook, we will cover some of its most useful and fun features, including:

* Templates
* Memory
* Working with APIs
* Chains
* Agents
* Vector stores

Let's start by importing some packages

In [1]:
! pip install --upgrade google-cloud-aiplatform
# LangChain
! pip install langchain langchain-experimental langchain[docarray]
! pip install -U langchain-google-vertexai
! pip install pypdf
! pip install pydantic==1.10.8
# Open source vector store
! pip install chromadb==0.3.26
! pip install typing-inspect==0.8.0 typing_extensions==4.5.0
# For dense vector representations of text
! pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 22.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 57.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.60.0
    Uninstalling google-cloud-aiplatform-1.60.0:
      Successfully uninstalled google-cloud-aiplatform-1.60.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires grpcio!=1.48.0,<2,>=1.33.1, but you have grpcio 1.48.0 which is incompatible.
google-cloud-pubsub 2.21.4 requires grpcio<2.0dev,>=1.51.3, but you have grpcio 1.48.0 which is incompatible.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you h

In [2]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import vertexai

vertexai.init()

In [2]:
# Utils
import time
from typing import List

# Langchain
import langchain
from pydantic import BaseModel

print(f"LangChain version: {langchain.__version__}")

# Vertex AI
from google.cloud import aiplatform
from langchain_google_vertexai import ChatVertexAI
from langchain.embeddings import VertexAIEmbeddings
from langchain_google_vertexai import VertexAI
from langchain.schema import HumanMessage, SystemMessage

print(f"Vertex AI SDK version: {aiplatform.__version__}")

LangChain version: 0.2.12
Vertex AI SDK version: 1.61.0


In [3]:
# We will use chat for some tasks
chat = ChatVertexAI(
    max_output_tokens=1024,
    temperature=0.2,
    top_p=0.8,
    top_k=40,
    verbose=True)

# And we will use the general text llm for others
llm = VertexAI(
    max_output_tokens=1024,
    temperature=0.2,
    top_p=0.8,
    top_k=40,
    verbose=True)

I0000 00:00:1723293223.946788    7462 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache


The simplest LangChain use is to create chats comprising of a `SystemMessage` and `HumanMessage`. This is similar to the `context` and `user_message` that we provide the LLM using the Python client libraries.

In [4]:
chat([HumanMessage(content="Hello")])

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


AIMessage(content=' Hi there! How can I help you today?', response_metadata={'is_blocked': False, 'errors': (), 'safety_attributes': [{'Finance': 0.1, 'Health': 0.3, 'Insult': 0.1, 'Sexual': 0.1}], 'grounding_metadata': {'citations': [], 'search_queries': []}, 'usage_metadata': {'candidates_billable_characters': 28.0, 'candidates_token_count': 10.0, 'prompt_billable_characters': 5.0, 'prompt_token_count': 1.0}}, id='run-1b6f4417-9f7e-47c1-b028-446d90ef9267-0', usage_metadata={'input_tokens': 1, 'output_tokens': 10, 'total_tokens': 11})

In [5]:
res = chat(
    [
        SystemMessage(
            content="You are an expert chef that thinks of imaginative recipies when people give you ingredients."
        ),
        HumanMessage(content="I have some kidney beans and tomatoes, what would be an easy lunch?"),
    ]
)

print(res.content)

 Kidney beans and tomatoes are a great combination for a quick and easy lunch. Here's a simple recipe you can try:

**Ingredients:**

- 1 can (15 ounces) kidney beans, rinsed and drained
- 1 can (14.5 ounces) diced tomatoes, undrained
- 1/2 cup chopped onion
- 1/2 cup chopped green bell pepper
- 1/4 cup chopped cilantro
- 1 teaspoon chili powder
- 1/2 teaspoon ground cumin
- 1/4 teaspoon salt
- 1/4 teaspoon black pepper
- 1/4 cup shredded cheese (optional)
- Tortillas, chips, or bread, for serving

**Instructions:**

1. In a large skillet, heat a little bit of oil over medium heat.
2. Add the onion and green bell pepper and cook until softened, about 5 minutes.
3. Add the kidney beans, tomatoes, chili powder, cumin, salt, and black pepper. Bring to a simmer and cook for 10 minutes, or until the beans are heated through.
4. Stir in the cilantro and cheese, if using.
5. Serve with tortillas, chips, or bread.

Enjoy your delicious and easy kidney bean and tomato lunch!


## Prompt templates

Templates are an abstraction that can help keep prompts modular and reusable. This can be especially important in large applications which may require long and varied prompts.

Templates may include few-short examples, instructions, or context.

In [6]:
# The template_string parameters sets the context for the ChatPromptTemplate

template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [7]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

The chat prompts are envisioned as a series of messages. Notice the `.messages` and `format_messages` methods in the following cells.

In [8]:
# Print out the template
prompt_template.messages

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'))]

In [9]:
#Let's check just the prompt
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [10]:
# Helpful method to keep track of a template's inputs
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In this simple example, we translate a customer e-mail into phonetic Glaswegian.

In [11]:
translator_style = """A translator that writes in phonetic Glaswegian.
"""

In [12]:
customer_email = """
This smashing little coffee maker is simply brilliant! \
I'm so pleased with how easy it is to use and how quickly it brews \
a cracking cup of coffee. \
I'm over the moon with this purchase and would highly recommend it \
to any other coffee lover looking for a top-notch brew every time.
"""

In [13]:
# The format_messages method sets up the task specified in the template
customer_messages = prompt_template.format_messages(
                    style=translator_style,
                    text=customer_email)

In [14]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)
print(customer_response.content)

 Aww, right, so, listen up, pal. This wee coffee maker is pure dead brilliant! It's a doddle tae use, and it makes a braw cuppa in nae time. Ah'm ower the moon wi' it, and Ah'd recommend it tae ony coffee lover lookin' fur a wee belter.


## Parsing outputs

LangChain makes it easy to return objects from the LLM in a format which we can use for further tasks (for example, adding an item of interest to a shopping cart, or providing a short list back to the LLM for additional questions).

Here is an example of parsing customer reviews of a three-course meal in a restaurant.

In [15]:
customer_review = """\
The excellent barbecue cauliflower starter left \
a lasting impression -- gorgeous presentation and flavors, really geared the tastebuds into action. \
Moving on to the main course, pretty great also. \
Delicious and flavorful chickpea and vegetable curry. They really nailed the buttery consistency, \
depth and balance of the spices. \
The dessert was a bit bland. I opted for a vegan chocolate mousse, \
hoping for a decadent and indulgent finale to my meal. \
It was very visually appealing but was missing the smooth, velvety \
texture of a great mousse.
"""

review_template = """\
For the input text, extract the following details: \
starter: How did the reviewer find the first course? \
Rate either Poor, Good, or Excellent. \
Do the same for the main course and dessert

Format the output as JSON with the following keys:
starter
main_course
dessert

text: {text}
"""



In [16]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='For the input text, extract the following details: starter: How did the reviewer find the first course? Rate either Poor, Good, or Excellent. Do the same for the main course and dessert\n\nFormat the output as JSON with the following keys:\nstarter\nmain_course\ndessert\n\ntext: {text}\n'))]


In [17]:
messages = prompt_template.format_messages(text=customer_review)
response = chat(messages, temperature=0.1)
print(response.content)

 ```json
{
  "starter": "Excellent",
  "main_course": "Good",
  "dessert": "Poor"
}
```


Though it looks like JSON, our output is actually a string type.

In [18]:
type(response.content)

str

This means we are unable to access values in this fashion (will result in an error):

In [20]:
response.content.get("main_course")

AttributeError: 'str' object has no attribute 'get'

This is where LangChain's parser comes in. Here, we import the `ResponseSchema` and `StructuredOutputParser`, which we use to define the format of the results from the LLM.

In [21]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

starter_schema = ResponseSchema(name="starter", description="Review of the starter")
main_course_schema = ResponseSchema(name="main_course", description="Review of the main course")
dessert_schema = ResponseSchema(name="dessert", description="Review of the dessert")

response_schemas = [starter_schema, main_course_schema, dessert_schema]

In [22]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [23]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"starter": string  // Review of the starter
	"main_course": string  // Review of the main course
	"dessert": string  // Review of the dessert
}
```


Now we can update our prior review template to include the format instructions

In [24]:
review_template_2 = """\
For the input text, extract the following details: \
starter: How did the reviewer find the first course? \
Rate either Poor, Good, or Excellent. \
Do the same for the main course and dessert

starter
main_course
dessert

text: {text}

{format_instructions}
"""
prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review,
                                format_instructions=format_instructions)

In [25]:
print(messages[0].content)

For the input text, extract the following details: starter: How did the reviewer find the first course? Rate either Poor, Good, or Excellent. Do the same for the main course and dessert

starter
main_course
dessert

text: The excellent barbecue cauliflower starter left a lasting impression -- gorgeous presentation and flavors, really geared the tastebuds into action. Moving on to the main course, pretty great also. Delicious and flavorful chickpea and vegetable curry. They really nailed the buttery consistency, depth and balance of the spices. The dessert was a bit bland. I opted for a vegan chocolate mousse, hoping for a decadent and indulgent finale to my meal. It was very visually appealing but was missing the smooth, velvety texture of a great mousse.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"starter": string  // Review of the starter
	"main_course": string  // Review of the

In [26]:
response = chat(messages)

Let's try it on the same review

Our response starts as an `AIMessage`

In [27]:
type(response)

langchain_core.messages.ai.AIMessage

Here we parse the `AIMessage` into a Python dictionary

In [28]:
output_dict = output_parser.parse(response.content)
output_dict

{'starter': 'Excellent', 'main_course': 'Good', 'dessert': 'Poor'}

Thanks to LangChain's parser, we now have a Python dictionary which we can use for further tasks, for example taking part of the response and using it as an input to another function / process etc.

In [29]:
type(output_dict)

dict

In [30]:
output_dict.get("main_course")

'Good'

## API chains

Another of LangChain's useful features is the ability to call external APIs within chains.

In this example, we use the `open-meteo.com` API to get weather reports.

In [31]:
from langchain.chains import APIChain
from langchain.chains.api import open_meteo_docs

llm = VertexAI(temperature=0)
chain = APIChain.from_llm_and_api_docs(
    llm,
    open_meteo_docs.OPEN_METEO_DOCS,
    verbose=True,
    limit_to_domains=["https://api.open-meteo.com/"],
)
chain.run(
    "How is the weather today in Edinburgh, Scotland, in Fahrenheit?"
    )

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new APIChain chain...
 https://api.open-meteo.com/v1/forecast?latitude=55.9533&longitude=-3.1883&hourly=temperature_2m&temperature_unit=fahrenheit&current_weather=true
{"latitude":55.96,"longitude":-3.18,"generationtime_ms":0.05900859832763672,"utc_offset_seconds":0,"timezone":"GMT","timezone_abbreviation":"GMT","elevation":69.0,"current_weather_units":{"time":"iso8601","interval":"seconds","temperature":"°F","windspeed":"km/h","winddirection":"°","is_day":"","weathercode":"wmo code"},"current_weather":{"time":"2024-08-10T12:30","interval":900,"temperature":66.3,"windspeed":26.7,"winddirection":260,"is_day":1,"weathercode":1},"hourly_units":{"time":"iso8601","temperature_2m":"°F"},"hourly":{"time":["2024-08-10T00:00","2024-08-10T01:00","2024-08-10T02:00","2024-08-10T03:00","2024-08-10T04:00","2024-08-10T05:00","2024-08-10T06:00","2024-08-10T07:00","2024-08-10T08:00","2024-08-10T09:00","2024-08-10T10:00","2024-08-10T11:00","2024-08-10T12:00","2024-08-10T13:00","2024-08-10T1

' The current temperature in Edinburgh, Scotland is 66.3 degrees Fahrenheit.'

### Wikipedia

We can combine the Wikipedia pip package and LangChain's Wikipedia API wrapper get query results from the encyclopedia.

In [32]:
!pip install wikipedia

I0000 00:00:1723293375.797769    7462 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork


  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=39acb4e749208a5fd7d311e511ba0fadb942e0489bfaa7803487a1f7e6a94d8a
  Stored in directory: /home/jupyter/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [33]:
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

wikipedia.run("To which bird family does the field sparrow belong?")

'Page: House sparrow\nSummary: The house sparrow (Passer domesticus) is a bird of the sparrow family Passeridae, found in most parts of the world. It is a small bird that has a typical length of 16 cm (6.3 in) and a mass of 24–39.5 g (0.85–1.39 oz). Females and young birds are coloured pale brown and grey, and males have brighter black, white, and brown markings. One of about 25 species in the genus Passer, the house sparrow is native to most of Europe, the Mediterranean Basin, and a large part of Asia. Its intentional or accidental introductions to many regions, including parts of Australasia, Africa, and the Americas, make it the most widely distributed wild bird.\nThe house sparrow is strongly associated with human habitation, and can live in urban or rural settings. Though found in widely varied habitats and climates, it typically avoids extensive woodlands, grasslands, polar regions, and hot, dry deserts far away from human development. For sustenance, the house sparrow routinely 

### Google search

In [34]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMRequestsChain, LLMChain

template = """Between >>> and <<< are the raw search result text from google.
Extract the answer to the question '{query}' or say "not found" if the information is not contained.
Use the format
Extracted:<answer or "not found">
>>> {requests_result} <<<
Extracted:"""

PROMPT = PromptTemplate(
    input_variables=["query", "requests_result"],
    template=template,
)


chain = LLMRequestsChain(llm_chain=LLMChain(llm=VertexAI(temperature=0), prompt=PROMPT))
question = "What are the official languages in Turkmenistan, and their alphabets?"
inputs = {
    "query": question,
    "url": "https://www.google.com/search?q=" + question.replace(" ", "+"),
}
chain(inputs)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'query': 'What are the official languages in Turkmenistan, and their alphabets?',
 'url': 'https://www.google.com/search?q=What+are+the+official+languages+in+Turkmenistan,+and+their+alphabets?',
 'output': ' Extracted: The official language of Turkmenistan is Turkmen, spoken by 77% of the population. The Turkmen language has been written with a version of the Latin alphabet based on Turkish since Turkmenistan declared independence in 1991.'}

## Memory

It is essential that LLMs keep some memory of the prior interactions in a chat to better inform their answers.

LangChain offers several approaches and features in this regard. For all details, see the [Memory](https://python.langchain.com/docs/modules/memory/) section of the documentation.

### ConversationBufferWindowMemory

Maintains a list of the interactions of the conversation over time, using the last K interactions. This can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large

In [35]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=3)

memory.save_context({"input": "Hi"},
                    {"output": "How are you?"})
memory.save_context({"input": "Fine thanks"},
                    {"output": "Great"})

memory.load_memory_variables({})

{'history': 'Human: Hi\nAI: How are you?\nHuman: Fine thanks\nAI: Great'}

### ConversationTokenBufferMemory

This feature instead keeps a buffer of recent interactions in memory based on token length,  rather than number of interactions.

In [36]:
from langchain.memory import ConversationTokenBufferMemory

memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "All alone, she dreams of the stars!"},
                    {"output": "As she should!"})
memory.save_context({"input": "Baking cookies today?"},
                    {"output": "Behold the cookies!"})
memory.save_context({"input": "Chatbots everywhere?"},
                    {"output": "Certainly!"})

In [37]:
memory.load_memory_variables({})

{'history': 'Human: All alone, she dreams of the stars!\nAI: As she should!\nHuman: Baking cookies today?\nAI: Behold the cookies!\nHuman: Chatbots everywhere?\nAI: Certainly!'}

### Conversation summaries

LangChain carries forward summaries of chat messages and flushes memory after a specified number of interactions or tokens.

Let's first look at using the former, `ConversationBufferWindowMemory`.

We set `verbose=True` to show the prompts and information carried forward by the LLM.

In [38]:
from langchain.memory import ConversationBufferWindowMemory

conversation_with_summary = ConversationChain(
    llm=VertexAI(temperature=0),
    # We set a low k=2, to only keep the last 2 interactions in memory
    memory=ConversationBufferWindowMemory(k=2),
    verbose=True
)
conversation_with_summary.predict(input="My favourite sport is fencing. Any tips for how I can go pro?")

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  warn_deprecated(




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: My favourite sport is fencing. Any tips for how I can go pro?
AI:

> Finished chain.


' Absolutely, here are some tips to help you become a professional fencer:\n\n**1. Master the basics:**\n   - Ensure you have a solid understanding of the fundamental techniques, including footwork, bladework, and body positioning.\n\n**2. Physical fitness:**\n   - Develop your overall fitness, including cardiovascular endurance, strength, and flexibility. Fencing requires quick movements and agility.\n\n**3. Practice regularly:**\n   - Consistent practice is crucial. Dedicate time each day to refine your skills and techniques.\n\n**4. Join a fencing club:**\n   - Training with experienced fencers can provide valuable insights and'

In [39]:
conversation_with_summary.predict(input="What equipment do I need?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: My favourite sport is fencing. Any tips for how I can go pro?
AI:  Absolutely, here are some tips to help you become a professional fencer:

**1. Master the basics:**
   - Ensure you have a solid understanding of the fundamental techniques, including footwork, bladework, and body positioning.

**2. Physical fitness:**
   - Develop your overall fitness, including cardiovascular endurance, strength, and flexibility. Fencing requires quick movements and agility.

**3. Practice regularly:**
   - Consistent practice is crucial. Dedicate time each day to refine your skills and techniques.

**4. Join a fencing club:**
   - Training with experienced fencers can provide 

" Here's a list of essential equipment you'll need for fencing:\n\n**1. Fencing weapon:**\n   - Choose a weapon based on your preference and style. The three main types are foil, épée, and saber.\n\n**2. Fencing mask:**\n   - A high-quality mask is crucial for protecting your face during bouts.\n\n**3. Fencing jacket:**\n   - The jacket provides protection for your torso and arms.\n\n**4. Fencing knickers:**\n   - These specialized pants offer flexibility and protection for your legs.\n\n**5. Fencing glove:**\n   - A well-fitting"

In [40]:
conversation_with_summary.predict(input="Who are the greats of the sport I can emulate?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: My favourite sport is fencing. Any tips for how I can go pro?
AI:  Absolutely, here are some tips to help you become a professional fencer:

**1. Master the basics:**
   - Ensure you have a solid understanding of the fundamental techniques, including footwork, bladework, and body positioning.

**2. Physical fitness:**
   - Develop your overall fitness, including cardiovascular endurance, strength, and flexibility. Fencing requires quick movements and agility.

**3. Practice regularly:**
   - Consistent practice is crucial. Dedicate time each day to refine your skills and techniques.

**4. Join a fencing club:**
   - Training with experienced fencers can provide 

" Here are some of the greatest fencers you can emulate to improve your skills and knowledge of the sport:\n\n**1. Valentina Vezzali (Italy):**\n   - One of the most decorated fencers in history, with multiple Olympic and World Championship titles in women's foil.\n\n**2. Alexander Romankov (Russia):**\n   - A legendary fencer known for his exceptional technique and achievements in men's foil.\n\n**3. Ed Zver (Russia):**\n   - A former Olympic champion and World Champion in men's saber, renowned for his speed and precision.\n\n**4"

In [41]:
# Since we have now passed k=2, the LLM will be unable to answer
conversation_with_summary.predict(input="What is my favourite sport?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What equipment do I need?
AI:  Here's a list of essential equipment you'll need for fencing:

**1. Fencing weapon:**
   - Choose a weapon based on your preference and style. The three main types are foil, épée, and saber.

**2. Fencing mask:**
   - A high-quality mask is crucial for protecting your face during bouts.

**3. Fencing jacket:**
   - The jacket provides protection for your torso and arms.

**4. Fencing knickers:**
   - These specialized pants offer flexibility and protection for your legs.

**5. Fencing glove:**
   - A well-fitting
Human: Who are the greats of the sport I can emulate?
AI:  Here are some of the greatest fencers you can emulate to impr

' I cannot answer that as the information is not available in the context.'

### ConversationSummaryBufferMemory

Ensures conversational memory up to a specified token length

In [42]:
from langchain.chains import ConversationChain

conversation_with_summary = ConversationChain(
    llm=llm,
    # Change max_token_limit here after running through the conversation
    memory=ConversationTokenBufferMemory(llm=llm, max_token_limit=600),
    verbose=True,
)
conversation_with_summary.predict(input="Hi, how are you?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, how are you?
AI:

> Finished chain.


' I am doing well, thank you for asking. How can I help you today?'

In [43]:
conversation_with_summary.predict(input="I'm learning the Rust programming language")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, how are you?
AI:  I am doing well, thank you for asking. How can I help you today?
Human: I'm learning the Rust programming language
AI:

> Finished chain.


" That's great! Rust is a powerful and versatile programming language that emphasizes safety, performance, and concurrency. It's particularly well-suited for building systems software, embedded systems, and high-performance applications. If you have any specific questions about Rust, feel free to ask."

In [44]:
conversation_with_summary.predict(input="What's the best book to help me?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, how are you?
AI:  I am doing well, thank you for asking. How can I help you today?
Human: I'm learning the Rust programming language
AI:  That's great! Rust is a powerful and versatile programming language that emphasizes safety, performance, and concurrency. It's particularly well-suited for building systems software, embedded systems, and high-performance applications. If you have any specific questions about Rust, feel free to ask.
Human: What's the best book to help me?
AI:

> Finished chain.


" There are several excellent books that can help you learn Rust. Here are a few highly recommended options:\n\n* **The Rust Programming Language** by Steve Klabnik and Carol Nichols: This is the official book from the Rust team and provides a comprehensive introduction to the language. It covers everything from basic syntax to advanced topics like memory management and concurrency.\n\n* **Rust in Action** by Tim McNamara: This book takes a more hands-on approach, with plenty of code examples and exercises. It's a great choice if you want to learn by doing.\n\n* **Programming Rust** by Jim Blandy: This book is known"

In [45]:
# Notice the buffer here is updated and clears the earlier exchanges
# Depending on how chatty the LLM is feeling, the token limit may have
# already been reached, and this cell will yield a generic response.
conversation_with_summary.predict(input="Wish me luck!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, how are you?
AI:  I am doing well, thank you for asking. How can I help you today?
Human: I'm learning the Rust programming language
AI:  That's great! Rust is a powerful and versatile programming language that emphasizes safety, performance, and concurrency. It's particularly well-suited for building systems software, embedded systems, and high-performance applications. If you have any specific questions about Rust, feel free to ask.
Human: What's the best book to help me?
AI:  There are several excellent books that can help you learn Rust. Here are a few highly recommended options:

* **The Rust Programming Language** by Steve Klabnik and Carol Nichols: Th

" Good luck with your Rust learning journey! Rust is a fantastic language, and I'm confident you'll find it both enjoyable and rewarding. If you have any further questions or need additional assistance, feel free to ask. I'm here to help you succeed."

The following cell should generate a reply that is clearly restricted to the general benefits of learning Haskell and missing the previous context of someone trying to learn Rust.

Run this cell, then go back to the Keep the conversation going with summaries cell and change `max_token_limit` to 700. Then re-run the entire conversation and notice how the model relates its ouptut about learning Haskell to the context of someone trying to learn Rust.

In [46]:
conversation_with_summary.predict(input="Would knowing Haskell help me?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, how are you?
AI:  I am doing well, thank you for asking. How can I help you today?
Human: I'm learning the Rust programming language
AI:  That's great! Rust is a powerful and versatile programming language that emphasizes safety, performance, and concurrency. It's particularly well-suited for building systems software, embedded systems, and high-performance applications. If you have any specific questions about Rust, feel free to ask.
Human: What's the best book to help me?
AI:  There are several excellent books that can help you learn Rust. Here are a few highly recommended options:

* **The Rust Programming Language** by Steve Klabnik and Carol Nichols: Th

' Yes, knowing Haskell can be beneficial for learning Rust. Haskell is a purely functional programming language that emphasizes immutability, referential transparency, and lazy evaluation. While Rust is not a purely functional language, it does incorporate some functional programming concepts, such as immutability and pattern matching. Additionally, both Rust and Haskell place a strong emphasis on type safety and memory management. Therefore, having a background in Haskell can provide you with a solid foundation for understanding and appreciating these aspects of Rust.'

## Chains

Complex applications will require chaining LLMs together, or with other components.

We will cover the following types of chains:

**Sequential chains**

**Router chains**

### LLMChain

An LLMChain simply provides a prompt to the LLM.

In [47]:
from langchain.chains import LLMChain

prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [48]:
chain = LLMChain(llm=llm, prompt=prompt)
product = "A saw for laminate wood"
chain.run(product)

' **Laminate Lumber Sawing Solutions**'

### Sequential chain

A sequential chain makes a series of calls to an LLM. It enables a pipeline-style workflow in which the output from one call becomes the input to the next.

The two types include:

* `SimpleSequentialChain`, where predictably each step has a single input and output, which becomes the input to the next step.

* `SequentialChain`, which allows for multiple inputs and outputs.

In [49]:
from langchain.chains import SimpleSequentialChain
from langchain.prompts import PromptTemplate

In [50]:
# This is an LLMChain to write a pitch for a new product
# Let's increase the temperature to allow some imagination

llm = VertexAI(temperature=0.7)
template = """You are an entrepreneur. Think of a ground breaking new product and write a short pitch.

Title: {title}
Entrepreneur: This is a pitch for the above product:"""
prompt_template = PromptTemplate(input_variables=["title"], template=template)
pitch_chain = LLMChain(llm=llm, prompt=prompt_template)

In [51]:
template = """You are a panelist on Dragon's Den. Given a \
description of the product, you are to explain why you think it will \
succeed or fail in the market.

Product pitch: {pitch}
Review by Dragon's Den panelist:"""
prompt_template = PromptTemplate(input_variables=["pitch"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

In [52]:
# This is the overall chain where we run these two chains in sequence.
overall_chain = SimpleSequentialChain(chains=[pitch_chain, review_chain], verbose=True)

In [53]:
review = overall_chain.run("Portable iced coffee maker")



> Entering new SimpleSequentialChain chain...
 **Introducing the Portable Iced Coffee Maker: The Coolest Way to Brew Your Favorite Drink!**

**Problem:** Iced coffee is a delicious and refreshing beverage, but it can be a hassle to make at home. Traditional methods require a lot of time and effort, and they often result in watered-down coffee.

**Solution:** The Portable Iced Coffee Maker is the perfect solution for coffee lovers who want to enjoy delicious iced coffee without all the hassle. This revolutionary new product brews iced coffee in just minutes, and it's so portable that you can take it with you wherever you go.

**Benefits:**

* **Quick and
 As a panelist on Dragon's Den, I would have several concerns about the Portable Iced Coffee Maker. While the idea of a portable iced coffee maker is appealing, there are several factors that could limit its success in the market.

**Competition:** The market for portable coffee makers is already quite crowded, with several establishe

### Router chain

A `RouterChain` dynamically selects the next chain to use for a given input.
This feature uses the `MultiPromptChain` to select then answer with the best-suited prompt to the question.

This can help a modular architecure, allowing the effective triaging of inputs between relevant prompt templates.

In [54]:
from langchain.chains.router import MultiPromptChain

korean_template = """
You are an expert in korean history and culture.
Here is a question:
{input}
"""

spanish_template = """
You are an expert in spanish history and culture.
Here is a question:
{input}
"""

chinese_template = """
You are an expert in Chinese history and culture.
Here is a question:
{input}
"""

In [55]:
prompt_infos = [
    {
        "name": "korean",
        "description": "Good for answering questions about Korean history and culture",
        "prompt_template": korean_template,
    },
    {
        "name": "spanish",
        "description": "Good for answering questions about Spanish history and culture",
        "prompt_template": spanish_template,
    },
     {
        "name": "chinese",
        "description": "Good for answering questions about Chinese history and culture",
        "prompt_template": chinese_template,
    },
]

In [56]:
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser

llm = VertexAI(temperature=0)

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [57]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [58]:
# Thanks to Deeplearning.ai for this template and for the
# Langchain short course at deeplearning.ai/short-courses/.

MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [59]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [60]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

Notice in the outputs the country of speciality is prefixed eg:
`chinese: {'input': ...`, denoting the routing to the correct expert.

In [61]:
chain.run("What was the Han Dynasty?")



> Entering new MultiPromptChain chain...
chinese: {'input': 'What was the Han Dynasty?'}
> Finished chain.


" The Han Dynasty (206 BCE–220 CE) was a period of imperial rule in China that followed the fall of the Qin Dynasty. It is considered one of the most prosperous and influential periods in Chinese history. The Han Dynasty was founded by Liu Bang, a peasant rebel who overthrew the Qin Dynasty. The Han Dynasty was divided into two periods: the Western Han Dynasty (206 BCE–9 CE) and the Eastern Han Dynasty (25–220 CE). The Western Han Dynasty was based in Chang'an (modern Xi'an), while the Eastern Han Dynasty was based in Luoyang."

In [62]:
chain.run("What are some of typical dishes in Catalonia?")



> Entering new MultiPromptChain chain...
spanish: {'input': 'What are some of typical dishes in Catalonia?'}
> Finished chain.


" **Catalan cuisine** is a Mediterranean cuisine that has been influenced by the region's history and geography. Some of the most typical dishes in Catalonia include:\n\n* **Pa amb tomàquet** (bread with tomato): This is a simple but delicious dish that is often served as a snack or appetizer. It is made with toasted bread that is rubbed with tomato and drizzled with olive oil.\n* **Esqueixada** (cod salad): This salad is made with shredded cod, tomatoes, onions, and peppers. It is often served with bread or crackers.\n* **Escudella i carn d'olla**"

In [63]:
chain.run("How would I greet a friend's parents in Korean?")



> Entering new MultiPromptChain chain...
korean: {'input': "How would I greet a friend's parents in Korean?"}
> Finished chain.


" To greet a friend's parents in Korean, you would use the following phrases:\n\n**For the father:**\n\n- 안녕하세요 (annyeonghaseyo): This is the most common way to greet someone in Korean. It can be used in both formal and informal settings.\n- 아저씨 (ajeossi): This is a respectful way to address an older man. It can be used for your friend's father or any other older man you meet.\n- 아버지 (abeoji): This is a more formal way to address your friend's father. It is typically used when you are meeting"

In [64]:
chain.run("Summarize Don Quixote in a short paragraph")



> Entering new MultiPromptChain chain...
spanish: {'input': 'Summarize Don Quixote in a short paragraph'}
> Finished chain.


' Don Quixote, written by Miguel de Cervantes Saavedra, is a classic Spanish novel that tells the story of Alonso Quijano, a hidalgo from La Mancha who reads so many chivalric romances that he loses his mind and decides to become a knight-errant, calling himself Don Quixote of La Mancha. He recruits a peasant, Sancho Panza, as his squire, and together they embark on a series of adventures, often mistaking everyday objects and people for enchanted beings and formidable enemies. Through their misadventures, Cervantes satirizes the chivalric romances popular at the time and explores themes of idealism, reality, and the nature'

In [65]:
# No specialist chain for carburetor advice; this
# will be handled as any other input by the foundational model
chain.run("How can I fix a carburetor?")



> Entering new MultiPromptChain chain...
None: {'input': 'How can I fix a carburetor?'}
> Finished chain.


' Assistant: Fixing a carburetor can be a complex task, but here are some general steps you can follow to troubleshoot and potentially fix a carburetor issue:\n\n1. **Identify the Problem**:\n   - Check for any visible signs of damage or leaks on the carburetor.\n   - Listen for any unusual noises or changes in engine performance.\n   - Look for any disconnected or loose hoses or cables.\n\n2. **Clean the Carburetor**:\n   - Remove the carburetor from the engine.\n   - Disassemble the carburetor and clean all the components thoroughly using a carburetor cleaner and a soft brush.\n   - Pay special attention'

## Agents and vectorstores

This final section of the notebook will cover some of LangChain's most fun and powerful features.

Agents have access to tools such as JSON, Wikipedia, Web Search, GitHub or Pandas Dataframes, and can access their capabilities depending on user input.

See [here](https://python.langchain.com/docs/integrations/toolkits/) for a full list of agent toolkits.

We will work with some data to perform data retrieval using the LLM with embeddings to match customer queries to products. This is known as Retrieval Augmentated Generation, or RAG.

We will use the Wayfair [WANDS](https://www.aboutwayfair.com/careers/tech-blog/wayfair-releases-wands-the-largest-and-richest-publicly-available-dataset-for-e-commerce-product-search-relevance) dataset of more than 42,000 products. Here are the steps:

* Download the data into a pandas dataframe and take a smaller 1,000-row sample set

* Merge then generate embeddings for the product titles and descriptions

* Prompt an LLM to retrieve details and relevant documents related to queries.

<img src="https://assets.wfcdn.com/im/01139917/resize-h800-w800%5Ecompr-r85/2315/231567967/Capricornus+3+Seater+Sofa.jpg" width="250"/> <img src="https://assets.wfcdn.com/im/07725066/resize-h800-w800%5Ecompr-r85/1584/158440119/Vancasso+BOMOOTIUR+Stoneware+Dinnerware+-+Set+of+18.jpg" width="250"/>


In [66]:
!wget -q https://raw.githubusercontent.com/wayfair/WANDS/main/dataset/product.csv

I0000 00:00:1723293532.171223    7462 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork


In [67]:
import pandas as pd
product_df = pd.read_csv("product.csv", sep='\t')

We will work with 1,000 items to avoid longer wait times for the embedding and look up processes.

In [68]:
product_df = product_df[:2000].dropna()

In [69]:
product_df.head(3)

,product_id,product_name,product_class,category hierarchy,product_description,product_features,rating_count,average_rating,review_count
0,0,solid wood platform bed,Beds,Furniture / Bedroom Furniture / Beds & Headboa...,"good , deep sleep can be quite difficult to ha...",overallwidth-sidetoside:64.7|dsprimaryproducts...,15.0,4.5,15.0
1,1,all-clad 7 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,"create delicious slow-cooked meals , from tend...",capacityquarts:7|producttype : slow cooker|pro...,100.0,2.0,98.0
2,2,all-clad electrics 6.5 qt . slow cooker,Slow Cookers,Kitchen & Tabletop / Small Kitchen Appliances ...,prepare home-cooked meals on any schedule with...,features : keep warm setting|capacityquarts:6....,208.0,3.0,181.0


In [70]:
len(product_df)

1269

In [71]:
# Reduce the df to columns of interest
product_df = product_df.filter(["product_id", "product_name", "product_description", "average_rating"], axis=1)

In [72]:
product_df.head(3)

,product_id,product_name,product_description,average_rating
0,0,solid wood platform bed,"good , deep sleep can be quite difficult to ha...",4.5
1,1,all-clad 7 qt . slow cooker,"create delicious slow-cooked meals , from tend...",2.0
2,2,all-clad electrics 6.5 qt . slow cooker,prepare home-cooked meals on any schedule with...,3.0


### Import and initialize pandas dataframe agent

These tools use the `langchain-experimental` pip package installed at the start of the notebook.

### Pandas agent

This agent allows us to interact with the dataframe using natural language. LangChain shows us the pandas queries it is composing to answer the questions.

In [73]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType

agent = create_pandas_dataframe_agent(VertexAI(temperature=0), product_df, verbose=True, allow_dangerous_code=True)

In [74]:
agent.run("how many rows are there?")



> Entering new AgentExecutor chain...
 Thought: The number of rows in a dataframe can be found using the `len()` function.
Action: python_repl_ast
Action Input: `len(df)`1269 Question: how many rows are there?
Thought: The number of rows in a dataframe can be found using the `len()` function.
Action: python_repl_ast
Action Input: `len(df)`1269 Question: how many rows are there?
Thought: The number of rows in a dataframe can be found using the `len()` function.
Action: python_repl_ast
Action Input: `len(df)`1269 Question: how many rows are there?
Thought: The number of rows in a dataframe can be found using the `len()` function.
Action: python_repl_ast
Action Input: `len(df)`1269 Question: how many rows are there?
Thought: The number of rows in a dataframe can be found using the `len()` function.
Action: python_repl_ast
Action Input: `len(df)`1269 Question: how many rows are there?
Thought: The number of rows in a dataframe can be found using the `len()` function.
Action: python_repl_

'Agent stopped due to iteration limit or time limit.'

In [75]:
agent.run("How many products have a rating of > 4?")



> Entering new AgentExecutor chain...
 Thought: I need to filter the dataframe to only include products with a rating greater than 4.
Action: [python_repl_ast]
Action Input: `df[df['average_rating'] > 4]`[python_repl_ast] is not a valid tool, try one of [python_repl_ast]. Question: How many products have a rating of > 4?
Thought: I need to filter the dataframe to only include products with a rating greater than 4.
Action: [python_repl_ast]
Action Input: `df[df['average_rating'] > 4]`[python_repl_ast] is not a valid tool, try one of [python_repl_ast]. Question: How many products have a rating of > 4?
Thought: I need to filter the dataframe to only include products with a rating greater than 4.
Action: [python_repl_ast]
Action Input: `df[df['average_rating'] > 4].shape[0]`[python_repl_ast] is not a valid tool, try one of [python_repl_ast]. Question: How many products have a rating of > 4?
Thought: I need to filter the dataframe to only include products with a rating greater than 4.
Act

'Agent stopped due to iteration limit or time limit.'

### CSV agent

We can also work directly on a .csv file

In [76]:
pd.DataFrame.to_csv(product_df, "data.csv")

In [77]:
from langchain_experimental.agents.agent_toolkits import create_csv_agent

agent = create_csv_agent(
    VertexAI(temperature=0),
    "data.csv",
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True,
)

In [78]:
agent.run("How many rows are there?")



> Entering new AgentExecutor chain...
 Thought: The shape of the dataframe should give me the number of rows.
Action: python_repl_ast
Action Input: df.shape(1269, 5) The first value in the tuple is the number of rows.
Final Answer: 1269

> Finished chain.


'1269'

In [80]:
agent.run("Do any product descriptions mention cedar wood? Output them as JSON")



> Entering new AgentExecutor chain...


ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: ` Thought: The question asks for product descriptions that mention cedar wood. I can use `df['product_description'].str.contains('cedar wood')` to find the rows that contain 'cedar wood' in the 'product_description' column. Then I can use `df[df['product_description'].str.contains('cedar wood')]` to filter the dataframe and only keep the rows that contain 'cedar wood' in the 'product_description' column. Finally, I can use `df.to_json(orient='records')` to convert the filtered dataframe to JSON format.
Action: python_repl_`

In [81]:
agent.run("What is the square root of all ratings for product names featuring sofas")



> Entering new AgentExecutor chain...
 Thought: The question asks for the square root of all ratings for product names featuring sofas. To answer this question, I need to first filter the dataframe to only include rows where the product name contains the word "sofa", then calculate the square root of the average_rating column for those rows.
Action: [python_repl_ast]
Action Input:
```python
sofa_df = df[df['product_name'].str.contains('sofa')]
sofa_df['average_rating_sqrt'] = sofa_df['average_rating'].apply(lambda x: x ** 0.5)
[python_repl_ast] is not a valid tool, try one of [python_repl_ast]. Question: What is the square root of all ratings for product names featuring sofas?
Thought: The question asks for the square root of all ratings for product names featuring sofas. To answer this question, I need to first filter the dataframe to only include rows where the product name contains the word "sofa", then calculate the square root of the average_rating column for those rows.
Action:

'Agent stopped due to iteration limit or time limit.'

### Vector stores

We will explore embeddings vectors and vector stores in more detail in [subsequent notebooks](rastringer.io.github.com/promptcraft). Let's see what's possible by concatenating our `product_title` and `product_description` columns and creating a text file from the result. We can then create embeddings and perform various retrieval and Q&A tasks.

We will use the open source [Chroma](https://docs.trychroma.com/) vector store.

In [82]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader

We will embed a `text_data` column, which will be a concatenation of `product_name` and `product_description`, since both columns provide useful contextual information.

In [83]:
product_df['text_data'] = product_df['product_name'] + " " + product_df['product_description']

In [84]:
product_df["text_data"]

0       solid wood platform bed good , deep sleep can ...
1       all-clad 7 qt . slow cooker create delicious s...
2       all-clad electrics 6.5 qt . slow cooker prepar...
3       all-clad all professional tools pizza cutter t...
4       baldwin prestige alcott passage knob with roun...
                              ...                        
1991    2 '' gel memory foam mattress topper the refre...
1993    0.5 '' polyester mattress pad this luxurious m...
1995    alwyn home two-sided 6.5 '' firm memory foam m...
1996    8 '' medium innerspring mattress if you want t...
1999    betances all in one bed blocker hypoallergenic...
Name: text_data, Length: 1269, dtype: object

In [85]:
# Save the "text_data" column to a text file
text_file_path = "combined_text_data.txt"
product_df['text_data'].to_csv(text_file_path, sep='\t', index=False, header=False)


In [86]:
# load the document and split it into chunks
loader = TextLoader("combined_text_data.txt")
documents = loader.load()

### Text splitter

Splitting text is common when working with LangChain and LLMs in general. This practice means we can feed large amounts of data to LLMs for parsing or embedding in chunks, or batches.

Ideally, we want to do so in a way that keeps meaningful chunks together. We will use the default recommended `RecursiveCharacterTextSplitter`. We specify a `chunk_size` and `chunk_overlap` to set an upper limit on the size and overlap between the splits / chunks.

In [87]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

docs = text_splitter.split_documents(documents)

In [88]:
len(docs)

528

In [89]:
from langchain.vectorstores import Chroma

# Clear any previous vector store
!rm -rf ./docs/chroma

I0000 00:00:1723293698.155377    7462 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork


In [ ]:
# Takes ~3 mins to run
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma.from_documents(docs, embedding_function)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-08-10 12:41:50.696189: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enab

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
query = "Is there a slow cooker?"
docs = db.similarity_search(query, 2)

In [ ]:
docs[0]

In [ ]:
query = "Recommend a durable door mat"
docs = db.similarity_search(query, 2)

In [ ]:
docs[0]

### Retrieval

A `Retriever` is a method for answering questions based on information in an index.

Here, we use `RetrievalQA` this ability with a question and answering chain.

In [ ]:
from langchain.chains import RetrievalQA

llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=1024,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=db.as_retriever()
)

### Prompt

In [ ]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. \
If you don't know the answer, just say that you don't know, \
don't try to make up an answer. Use three sentences maximum. \
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)


In [ ]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=db.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [ ]:
question = "Can you recommend comfortable bed sheets?"
result = qa_chain({"query": question})
result["result"]

In [ ]:
question = "How about a Persian-style rug for my living room."
result = qa_chain({"query": question})
result["result"]

## Summary

In this whirlwind tour of some of LangChain's features, we covered:

* Memory
* Chains
* Agents
* Vector stores

LangChain is a fast-evolving project. To explore more features and keep up-to-date with developments, please see the [website](https://www.langchain.com/) or [Python documentation](https://python.langchain.com/docs/get_started/introduction).

With thanks to Harrison Chase and the excellent LangChain courses at [deeplearning.ai](https://deeplearning.ai/short-courses)